In [1]:
import os
import shutil
import random

source_base_dir = '/kaggle/input/image-pro/Image_Processing'  


base_dir = '/kaggle/working/'


original_dir = os.path.join(source_base_dir, 'Original')
pixelated_dir = os.path.join(source_base_dir, 'Pixelated')


train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')


os.makedirs(os.path.join(train_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Pixelated'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Pixelated'), exist_ok=True)


split_ratio = 0.8

def split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio):
    original_files = os.listdir(original_dir)
    pixelated_files = os.listdir(pixelated_dir)

    
    paired_files = [(file, file) for file in original_files if file in pixelated_files]

    random.shuffle(paired_files)
    split_index = int(len(paired_files) * split_ratio)
    train_files = paired_files[:split_index]
    test_files = paired_files[split_index:]

    for original_file, pixelated_file in train_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(train_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(train_dir, 'Pixelated', pixelated_file))
    
    for original_file, pixelated_file in test_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(test_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(test_dir, 'Pixelated', pixelated_file))

split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio)

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


2024-06-14 09:39:41.114989: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 09:39:41.115165: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 09:39:41.310015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/working/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/kaggle/working/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/working/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 6308 images belonging to 2 classes.
Found 1576 images belonging to 2 classes.
Found 1972 images belonging to 2 classes.


In [4]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


197/197 ━━━━━━━━━━━━━━━━━━━━ 561s 3s/step - accuracy: 0.5997 - loss: 0.9541 - val_accuracy: 0.7519 - val_loss: 0.5199
Epoch 2/10
  1/197 ━━━━━━━━━━━━━━━━━━━━ 5:52 2s/step - accuracy: 0.7812 - loss: 0.4319

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7812 - loss: 0.4319 - val_accuracy: 0.7500 - val_loss: 0.4493
Epoch 3/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 544s 3s/step - accuracy: 0.7625 - loss: 0.4950 - val_accuracy: 0.7526 - val_loss: 0.5303
Epoch 4/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7500 - loss: 0.4809 - val_accuracy: 1.0000 - val_loss: 0.3300
Epoch 5/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 543s 3s/step - accuracy: 0.7954 - loss: 0.4365 - val_accuracy: 0.6652 - val_loss: 0.5984
Epoch 6/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6562 - loss: 0.5666 - val_accuracy: 0.8750 - val_loss: 0.4248
Epoch 7/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 539s 3s/step - accuracy: 0.7957 - loss: 0.4438 - val_accuracy: 0.7615 - val_loss: 0.5357
Epoch 8/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7500 - loss: 0.4358 - val_accuracy: 1.0000 - val_loss: 0.1252
Epoch 9/10
197/197 ━━━━━━━━━━━━━━━━━━━━ 534s 3s/step - accuracy: 0.8121 - loss: 0.4061 - val_accuracy: 0.7494 - 

In [6]:
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


61/61 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.7800 - loss: 0.4834
Test Accuracy: 79.30%


In [7]:
model.save('pixelated_detection_model.h5')
